 <h1 style="text-align: center; color: RED">İŞ BİRLİKÇİ FİLTRELEME</h1>

Bu başlık altında Item-Based, User-Based ve Model-Based Collaborative Filtering konuları ele alınacaktır.

 <h2 style="text-align: center; color: RED">1-ITEM-BASED COLLABORATIVE FILTERING</h2>

Item benzerliği üzerinden öneriler yapılır. Örneğin bir kişi bir film izlemiş olsun ve bu filmi beğenmiş olsun bu filme benzer beğenilme yapısı gösteren filmler önerilir.İçerik temelli filtrelemeden farkı ürünlerin içeriğinin benzerliği değil o ürünün beğenilme yapısına benzer beğeni yapısı olan ürünlere bakılmasıdır.

Peki bu benzer beğenilme yapısı ne demektir?

Elimizde beğendiğimiz bir film olsun.
![Test](datasets/20.png)
Tablo üzerinden  beğendiğimiz film ile aynı beğenilme yapısını gösteren film işaretlenmiştir.

İş birlikçilik ne demek?

Tablo üzerindeki bir topluluk yani kullanıcılar bir konuda fikir birliğine varmış gözüküyor. Yani Aşkı Memnu ile Movien filmlerine verdikleri puan dağılımları birbirne benzerdir. Dolayısıyla bu iki filmin beğenilme alışkanlıkları arasında bir korelasyon var gibi gözükmektedir. Bu iki filmin beğenilme alışkanlıklarını korelasyon gibi bir metrikle tanımlayabiliriz.Odağımız kullanıcılar değil sütunda bulunan filmlerdir.


 <h3 style="text-align: center; color: RED">UYGULAMA</h3>

<span style="color: blue; font-weight: bold;">1-)İş Problemi</span>


Online film izleme platformu iş birlikçi filtreleme yöntemi ile bir öneri sistemi geliştirmek istemektedir. İçerik temelli öneri sitemini deneyen şirket topluluğun kanaatlerini barındıracak şekilde öneriler geliştirmek istemektedir. Kullacılar bir filmi beğendiğinde o filme benzer beğenilme örüntüsüne sahip olan diğer filmler önerilmek istenmektedir.

<span style="color: blue; font-weight: bold;">2-)Veri Seti Hikayesi</span>

Veri seti MovieLens tarafından sağlanmıştır.İçerisinde filmler ve bu filmlere verilen puanları barındırmaktadır. Veri seti yaklaşık 27000 film için yaklaşık 2000000 derecelendirme içermektedir.2 ayrı veri seti kullanıcaz.

<span style="color: blue; font-weight: bold;">3-)Veri Setinin Hazırlanması</span>

In [26]:
import pandas as pd
pd.set_option('display.max_columns', 500)


In [28]:
movie = pd.read_csv("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\5. Recommendation Systems\\datasets\\movie_lens_dataset\\movie.csv")
movie.head()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [30]:
rating = pd.read_csv("C:\\Users\\ASLI\\Desktop\\Data Scientist Bootcamp\\5. Recommendation Systems\\datasets\\movie_lens_dataset\\rating.csv")
rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [32]:
df = movie.merge(rating, how="left", on="movieId")#ortak sütun üzerinden birleştirdik
df.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,3.0,4.0,1999-12-11 13:36:47
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,6.0,5.0,1997-03-13 17:50:52
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,8.0,4.0,1996-06-05 13:37:51
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,4.0,1999-11-25 02:44:47
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,11.0,4.5,2009-01-02 01:13:41


Veri setini incelediğimizde bir userId yani kullanıcı belli bir tarihte bir filme oy veriyor. Daha sonra başka kullanıcı aynı filme başka tarihte tekrar oy veriyor.Yani movieıd ve filmler çoklama durumunda.Amacımız bir fikme benzer beğenilme davranışı olan filmleri getirmek.

<span style="color: blue; font-weight: bold;">4-)User Movie Df'inin Oluşturulması</span>

In [40]:
df['title'].nunique()

27262

In [42]:
df['title'].value_counts().head()#her filme verilen puan sayısı

title
Pulp Fiction (1994)                 67310
Forrest Gump (1994)                 66172
Shawshank Redemption, The (1994)    63366
Silence of the Lambs, The (1991)    63299
Jurassic Park (1993)                59715
Name: count, dtype: int64

Veri setine baktığımızda çok yüksek puanlaması olan filmlerde var cok az puanlaması olan filmlerde . Biz bu kısımda bir eşik değer belirleyerek puanlaması az olan filmleri çalışmamızın dışında bırakıcaz. Çünkü df yapısnı oluştururken cok fazla film olması çalışma performansı açısından uzun süre almasına sebep olacaktır. 

In [45]:
comment_counts = pd.DataFrame(df['title'].value_counts())#her film için kacar tane rate var

In [47]:
comment_counts.head()

,count
title,
Pulp Fiction (1994),67310
Forrest Gump (1994),66172
"Shawshank Redemption, The (1994)",63366
"Silence of the Lambs, The (1991)",63299
Jurassic Park (1993),59715


In [51]:
comment_counts[comment_counts['count']<=1000]#1000 den az rate alan filmler

,count
title,
"Bear, The (Ours, L') (1988)",999
Rosewood (1997),999
Ted (2012),999
One Night at McCool's (2001),999
Marked for Death (1990),998
...,...
Rapture (Arrebato) (1980),1
"Education of Mohammad Hussein, The (2013)",1
Satanas (2007),1


In [53]:
comment_counts[comment_counts['count']<=1000].index#az rate sahip olan filmleirn isimleri

Index(['Bear, The (Ours, L') (1988)', 'Rosewood (1997)', 'Ted (2012)',
       'One Night at McCool's (2001)', 'Marked for Death (1990)',
       'Three to Tango (1999)', 'Adam's Rib (1949)',
       'I Now Pronounce You Chuck and Larry (2007)',
       'Italian for Beginners (Italiensk for begyndere) (2000)',
       'Husbands and Wives (1992)',
       ...
       'Satan's Sword (Daibosatsu tôge) (1960)',
       'Blind Massage (Tui na) (2014)', 'Prêt à tout (2014)',
       'Ditchdigger's Daughters, The (1997)', 'A.K. (1985)',
       'Rapture (Arrebato) (1980)',
       'Education of Mohammad Hussein, The (2013)', 'Satanas (2007)',
       'Psychosis (2010)', 'Innocence (2014)'],
      dtype='object', name='title', length=24103)

In [55]:
rare_movies = comment_counts[comment_counts['count']<=1000].index#az rate sahip olan filmleirn isimleri

In [57]:
common_movies = df[~df['title'].isin(rare_movies)]#az rate sahip olanların dışındaki filmleri getirdik.

In [59]:
common_movies.shape

(17766015, 6)

In [61]:
common_movies['title'].nunique()#27 bin filmden 3159 tane kaldı

3159

user_movie_df oluştururken istediğimiz yapıy aulaşmak için satırlarda kullanıcılar sutunlarda filmler ve kesişimlerinde aldıkları oy sayısı olmalı

In [63]:
user_movie_df = common_movies.pivot_table(index='userId', columns='title', values='rating')

In [67]:
user_movie_df.head()

title,"'burbs, The (1989)",(500) Days of Summer (2009),*batteries not included (1987),...And Justice for All (1979),10 Things I Hate About You (1999),"10,000 BC (2008)",101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),102 Dalmatians (2000),12 Angry Men (1957),12 Years a Slave (2013),127 Hours (2010),13 Going on 30 (2004),"13th Warrior, The (1999)",1408 (2007),15 Minutes (2001),16 Blocks (2006),17 Again (2009),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),"2 Fast 2 Furious (Fast and the Furious 2, The) (2003)","20,000 Leagues Under the Sea (1954)",200 Cigarettes (1999),2001: A Space Odyssey (1968),2010: The Year We Make Contact (1984),2012 (2009),2046 (2004),21 (2008),21 Grams (2003),21 Jump Street (2012),24 Hour Party People (2002),25th Hour (2002),27 Dresses (2008),28 Days (2000),28 Days Later (2002),28 Weeks Later (2007),3 Ninjas (1992),3-Iron (Bin-jip) (2004),30 Days of Night (2007),300 (2007),3000 Miles to Graceland (2001),"39 Steps, The (1935)",3:10 to Yuma (2007),40 Days and 40 Nights (2002),"40-Year-Old Virgin, The (2005)","400 Blows, The (Les quatre cents coups) (1959)",42 Up (1998),48 Hrs. (1982),50 First Dates (2004),50/50 (2011),54 (1998),"6th Day, The (2000)","7th Voyage of Sinbad, The (1958)",8 1/2 (8½) (1963),8 Heads in a Duffel Bag (1997),8 Mile (2002),8MM (1999),9 (2009),9 1/2 Weeks (Nine 1/2 Weeks) (1986),"A-Team, The (2010)",A.I. Artificial Intelligence (2001),AVP: Alien vs. Predator (2004),AVPR: Aliens vs. Predator - Requiem (2007),About Last Night... (1986),About Schmidt (2002),About a Boy (2002),"Absent-Minded Professor, The (1961)",Absolute Power (1997),"Abyss, The (1989)",Accepted (2006),"Accidental Tourist, The (1988)","Accused, The (1988)",Ace Ventura: Pet Detective (1994),Ace Ventura: When Nature Calls (1995),Across the Universe (2007),Adaptation (2002),Addams Family Values (1993),"Addams Family, The (1991)",Addicted to Love (1997),"Adjustment Bureau, The (2011)",Adventureland (2009),Adventures in Babysitting (1987),"Adventures of Baron Munchausen, The (1988)","Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)","Adventures of Milo and Otis, The (Koneko monogatari) (1986)","Adventures of Pinocchio, The (1996)","Adventures of Priscilla, Queen of the Desert, The (1994)","Adventures of Robin Hood, The (1938)","Adventures of Rocky and Bullwinkle, The (2000)","Adventures of Tintin, The (2011)",Aeon Flux (2005),"Affair to Remember, An (1957)",Affliction (1997),"African Queen, The (1951)",After Hours (1985),"Age of Innocence, The (1993)",Agnes of God (1985),"Aguirre: The Wrath of God (Aguirre, der Zorn Gottes) (1972)",Air America (1990),Air Bud (1997),Air Force One (1997),Airheads (1994),Airplane II: The Sequel (1982),Airplane! (1980),Airport (1970),Akira (1988),Aladdin (1992),Aladdin and the King of Thieves (1996),Alexander (2004),Alfie (2004),Ali (2001),Alice in Wonderland (1951),Alice in Wonderland (2010),Alien (1979),Alien Nation (1988),Alien: Resurrection (1997),Aliens (1986),Alien³ (a.k.a. Alien 3) (1992),Alive (1993),All About Eve (1950),All About My Mother (Todo sobre mi madre) (1999),All Dogs Go to Heaven (1989),All Dogs Go to Heaven 2 (1996),All Quiet on the Western Front (1930),All That Jazz (1979),All the President's Men (1976),All the Pretty Horses (2000),Almost Famous (2000),Along Came Polly (2004),Along Came a Spider (2001),"Alphaville (Alphaville, une étrange aventure de Lemmy Caution) (1965)",Altered States (1980),Always (1989),Amadeus (1984),Amateur (1994),"Amazing Spider-Man, The (2012)","Amelie (Fabuleux destin d'Amélie Poulain, Le) (2001)",America's Sweethearts (2001),American Beauty (1999),American Gangster (2007),American Gigolo (1980),American Graffiti (1973),American History X (1998),American Hustle (2013),American Movie (1999),American Pie (1999),American Pie 2 (2001),"American President, The (1995)",American Psycho (2000),American Splendor (2003),"American Tail, An (1986)","American Tail: Fievel Goes West, An (1991)",Amer

In [69]:
user_movie_df.shape

(138493, 3159)

<span style="color: blue; font-weight: bold;">5-)Item-Based Film Önerilerinin Yapılması</span>

In [73]:
movie_name='Matrix, The (1999)'#veri setinden bir film sectim
movie_name = user_movie_df[movie_name]#veri sertinden bu filmin sutunu sectik

In [75]:
movie_name

userId
1.0         NaN
2.0         NaN
3.0         5.0
4.0         NaN
5.0         NaN
           ... 
138489.0    4.0
138490.0    NaN
138491.0    NaN
138492.0    5.0
138493.0    4.5
Name: Matrix, The (1999), Length: 138493, dtype: float64

Bu kısımda sectiğim film ile benzer beğenilme yapısı olan diğer filmleri seciçez. Bunu da seçtiğimizfilm ile diğer filmler arasındaki korelasyona bakarak yapıcaz.

In [ ]:
user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)#en cok benzerlik gösteren 10 film

In [78]:
movie_name=pd.Series(user_movie_df.columns).sample(1).values[0]

* Bu kod, user_movie_df adlı bir DataFrame'in sütun isimlerinden rastgele bir tane seçip, bunu movie_name adlı değişkene atar.

1. user_movie_df.columns
Bu ifade, user_movie_df DataFrame'inin sütun adlarını verir.

2. pd.Series(user_movie_df.columns)
Bu, sütun isimlerini bir Pandas Series nesnesine çevirir.

3. .sample(1)
Bu ifade, bu Series içinden rastgele bir eleman seçer.
sample(1) → 1 tane örnek seç demek.

4. .values[0]
.values, seçilen örneği NumPy dizisine çevirir.
[0] ifadesi ise bu dizideki ilk (ve burada tek) elemanı alır.

In [82]:
#eğer filmin tam adını bilmiyorsak sadece bir keyworda göre film arayacaksan bu fonksiyonu kullanabiliriz.
def check_film(keyword, user_movie_df):
    return [col for col in user_movie_df.columns if keyword in col]

check_film("Insomnia", user_movie_df)

['Insomnia (1997)', 'Insomnia (2002)']

<span style="color: blue; font-weight: bold;">6-)Çalışma Scriptinin Hazırlanması</span>

In [ ]:
def create_user_movie_df():
    import pandas as pd
    movie = pd.read_csv('datasets/movie_lens_dataset/movie.csv')
    rating = pd.read_csv('datasets/movie_lens_dataset/rating.csv')
    df = movie.merge(rating, how="left", on="movieId")
    comment_counts = pd.DataFrame(df["title"].value_counts())
    rare_movies = comment_counts[comment_counts["title"] <= 1000].index
    common_movies = df[~df["title"].isin(rare_movies)]
    user_movie_df = common_movies.pivot_table(index=["userId"], columns=["title"], values="rating")
    return user_movie_df

user_movie_df = create_user_movie_df()


def item_based_recommender(movie_name, user_movie_df):
    movie_name = user_movie_df[movie_name]
    return user_movie_df.corrwith(movie_name).sort_values(ascending=False).head(10)

item_based_recommender("Matrix, The (1999)", user_movie_df)

movie_name = pd.Series(user_movie_df.columns).sample(1).values[0]

item_based_recommender(movie_name, user_movie_df)